In [2]:
:dep ndarray = "0.15.6"
:dep ndarray-rand = "0.14.0"
:dep showata = { version = "0.3.2", features=["show_ndarray"]}
//:dep polars = { version = "0.29.0", features=["ndarray"]}

In [3]:
use ndarray::{Array, Array2, stack, concatenate};
use ndarray_rand::RandomExt;
use ndarray_rand::rand_distr::Uniform;
use ndarray::prelude::*;

In [4]:
use std::f64;

#[derive(Debug)]
enum ActivationFunction {
    Tanh,
}

impl ActivationFunction {
    fn activate(&self, x: f64) -> f64 {
        match self {
            ActivationFunction::Tanh => self.tanh(x),
        }
    }
    
    fn derivative(&self, x: f64) -> f64 {
        match self {
            ActivationFunction::Tanh => self.tanh_derivative(x),
        }
    }
    
    fn tanh(&self, x: f64) -> f64 {
        x.tanh()
    }
    
    fn tanh_derivative(&self, x: f64) -> f64 {
        1.0 - self.tanh(x).powi(2)
    }
}



In [5]:
#[derive(Debug)]
struct Layer {
    len_inputs: usize,
    neurons: usize,
    function: ActivationFunction,
    weights: Array2<f64>,
    input: Option<Array2<f64>>,
    net: Option<Array2<f64>>,
    output: Option<Array2<f64>>,
    idx: Option<usize>,
}

impl Layer {
    fn new(len_inputs: usize, neurons: usize, function: ActivationFunction) -> Self {
        let shape = (neurons, len_inputs + 1);
        let weights = Array::random(shape, Uniform::new(-0.5, 0.5));

        Layer {
            len_inputs: len_inputs,
            neurons: neurons,
            function: function,
            weights: weights,
            input: None,
            net: None,
            output: None,
            idx: None,
        }
    }

    fn forward(&mut self, layer_input: &Array2<f64>) -> Array2<f64> {
        self.input = Some(layer_input.clone());
        self.net = Some(self.input.as_ref().unwrap().dot(&self.weights.t()));
        self.output = Some(self.net.as_ref().unwrap().mapv(|x| self.function.activate(x)));
        self.output.clone().unwrap()
    }

    
    fn backward(
        &mut self, 
        alpha: f64,
        last: bool,
        previous_delta: Option<&Array2<f64>>,
        previous_weight: Option<&Array2<f64>>,
        error: Option<&Array2<f64>>
    ) -> (Array2<f64>, Array2<f64>) {
        
        let delta = if last {
            let error = error.unwrap();
            error * self.net.as_ref().unwrap().mapv(|x| self.function.derivative(x))
        } else {
            let previous_delta = previous_delta.unwrap();
            let previous_weight = previous_weight.unwrap();
            let delta = previous_delta.dot(previous_weight).slice(s![.., 1..]).to_owned();
            delta * self.net.as_ref().unwrap().mapv(|x| self.function.derivative(x))
        };
        
        
        self.weights = delta.t().dot(self.input.as_ref().unwrap()) * alpha + &self.weights;
        
        (delta, self.weights.to_owned())
    }



    fn set_idx(&mut self, idx: usize) {
        self.idx = Some(idx);
    }
}

In [6]:
#[derive(Debug)]
struct NeuralNetwork {
    layers: Vec<Layer>,
    all_mse: Vec<f64>,
}

impl NeuralNetwork {
    fn new(mut layers: Vec<Layer>) -> Self {
        for (idx, layer) in layers.iter_mut().enumerate() {
            layer.set_idx(idx + 1);
        }
        
        NeuralNetwork {
            layers,
            all_mse: Vec::new(),
        }
    }

    fn forward(&mut self, x_input: &Array2<f64>) -> Array2<f64> {
        let mut input_layer = concatenate![Axis(1), Array::from_shape_vec((1, 1), vec![1.0]).unwrap(), x_input.clone()];
        let mut output: Array2<f64> = Array::zeros((0, 0));

        for layer in &mut self.layers {
            output = layer.forward(&input_layer);
            input_layer = concatenate![Axis(1), Array::from_shape_vec((1, 1), vec![1.0]).unwrap(), output];
        }

        output
    }

    fn backward(&mut self, alpha: f64, error: &Array2<f64>) {
        let mut previous_delta = None;
        let mut previous_weight = None;
        let mut last = true;
        
        for layer in self.layers.iter_mut().rev() {
            let (delta, weights) = layer.backward(alpha, last, previous_delta.as_ref(), previous_weight.as_ref(), Some(&error));
            last = false;
            previous_delta = Some(delta);
            previous_weight = Some(weights);
        }
    }

    fn predict(&mut self, x: &Array2<f64>) -> Array2<f64> {
        self.forward(x)
    }
}

In [617]:
let weights_1 = array![[0.2, 0.4, 0.5],[0.3, 0.6, 0.7],[0.4, 0.8, 0.3]];
let weights_2 = array![[-0.7, 0.6, 0.2, 0.7],[-0.3, 0.7, 0.2, 0.8]];
let weights_3 = array![[0.1, 0.8, 0.5]];


let mut nn = NeuralNetwork::new(vec![
    Layer::new(2, 3, ActivationFunction::Tanh),
    Layer::new(3, 2, ActivationFunction::Tanh),
    Layer::new(2, 1, ActivationFunction::Tanh),
]);

nn.layers[0].weights = weights_1;
nn.layers[1].weights = weights_2;
nn.layers[2].weights = weights_3;

let x_inputs = array![[0.3, 0.7]];
let out = nn.forward(&x_inputs);
out


[[0.5763660324621392]], shape=[1, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2

In [618]:
nn.backward(0.05, &array![[-2.0]]);
nn.predict(&x_inputs)

[[0.403346832835286]], shape=[1, 1], strides=[1, 1], layout=CFcf (0xf), const ndim=2

In [9]:
use polars::prelude::*;
let file_path = "train.csv";

    // Ler o arquivo CSV para um DataFrame
let df: DataFrame = CsvReader::from_path(file_path)
    .unwrap()
    .infer_schema(None)
    .has_header(true)
    .finish()
    .unwrap();

// Exibir o DataFrame

In [110]:
:dep ndarray-csv = {version = "0.4.1"}
:dep csv

In [10]:
let ndarray = df.to_ndarray::<Float64Type>().unwrap();

In [33]:
let mut z = Array::zeros((1, ndarray.shape()[1]-1));
for linha in ndarray.slice(s![.., 1..]).to_owned().axis_iter(Axis(0)) {
    z.assign(&linha);
}
z

[[0.0, 0.0, 0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0, 0.0, 0.0]], shape=[1, 784], strides=[784, 1], layout=CFcf (0xf), const ndim=2

In [18]:
let y_train = ndarray.slice(s![.., ..1]);
let x_train = ndarray.slice(s![.., 1..]).to_owned()

Error: The variable `y_train` has type `ArrayBase<ndarray::ViewRepr<&f64>, Dim<[usize; _]>>` which cannot be persisted.
You might be able to fix this by creating a `Box<dyn YourType>`. e.g.
let v: Box<dyn core::fmt::Debug> = Box::new(foo());
Alternatively, you can prevent evcxr from attempting to persist
the variable by wrapping your code in braces.

In [ ]:
ndarray.slice(s![..10, 1..]).to_owned().iter().map(|x| x*2.0).collect::<Vec<f64>>()

In [7]:

let mut nn = NeuralNetwork::new(vec![
    Layer::new(784, 20, ActivationFunction::Tanh),
    Layer::new(20, 20, ActivationFunction::Tanh),
    Layer::new(20, 10, ActivationFunction::Tanh),
]);




In [37]:
nn.layers[0].weights

[[0.0678924763507549, 0.026817238404717614, 0.4600452324269091, 0.29865287936780915, 0.3867389105412007, ..., -0.27152510334178026, 0.21817470976871367, -0.20259140072437098, 0.011645148821384455, -0.1651635784738379],
 [-0.41745129804815706, 0.23219777172019063, -0.4920782069601646, -0.32307680363188296, -0.3302317081543218, ..., 0.2843282650289085, -0.0977307901032689, -0.24252542365813423, 0.2037530724150347, 0.41597954984659724],
 [0.11248100199905497, -0.1637814672380895, -0.01522655292539632, 0.38826067562455724, 0.37865720563358507, ..., 0.2041100591086833, 0.017731740825965847, -0.33483931587481486, 0.1493255397312827, -0.46575563531640984],
 [0.20724052580551833, 0.17076519378053479, 0.48233035200533037, -0.46517426953018237, 0.3295027846615277, ..., 0.30563897973247145, 0.24037684156947337, 0.34366546702343514, -0.456945300156111, -0.19814285605318016],
 [0.01569383857754314, -0.34392953859054987, 0.15016157660280105, -0.0519266977240358, 0.028716919447639988, ..., -0.1695827

In [34]:
nn.forward(&z)

[[-0.8575975543026888, 0.7418844757167533, 0.008276932948479847, 0.30835947317996804, -0.9020352534626893, 0.7557398196992497, 0.7035063310380174, -0.7711807842923268, 0.5960649283081518, 0.9926227474377259]], shape=[1, 10], strides=[1, 1], layout=CFcf (0xf), const ndim=2

In [22]:
let weights = Array::random((2,3), Uniform::new(-0.5, 0.5));
weights

[[-0.2963793352136983, -0.15052976608205304, -0.27497275043491043],
 [-0.19151175794119135, 0.1644954224317805, -0.4306283995204303]], shape=[2, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2

In [21]:
fn ndarray_to_vec2d(arr: &Array2<f64>) -> Vec<Vec<f64>> {
    let mut vec2d: Vec<Vec<f64>> = Vec::new();
    
    for row in arr.outer_iter() {
        let row_vec: Vec<f64> = row.iter().cloned().collect();
        vec2d.push(row_vec);
    }
    
    vec2d
}

In [24]:
let a = ndarray_to_vec2d(&weights);
a

[[-0.2963793352136983, -0.15052976608205304, -0.27497275043491043], [-0.19151175794119135, 0.1644954224317805, -0.4306283995204303]]

In [18]:
fn vec2d_to_ndarray(vec2d: Vec<Vec<f64>>) -> Array2<f64> {
    let rows = vec2d.len();
    let cols = vec2d[0].len();
    
    let mut arr = Array::zeros((rows, cols));
    
    for (i, row) in vec2d.iter().enumerate() {
        for (j, &element) in row.iter().enumerate() {
            arr[[i, j]] = element;
        }
    }
    
    arr
}


In [19]:
let b = vec2d_to_ndarray(a);
b

[[-0.26562747473365445, -0.26194153720101476, -0.11753596474459416],
 [-0.05613611233936222, 0.3936101608844258, 0.3450691782536863]], shape=[2, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2

In [25]:
weights

[[-0.2963793352136983, -0.15052976608205304, -0.27497275043491043],
 [-0.19151175794119135, 0.1644954224317805, -0.4306283995204303]], shape=[2, 3], strides=[3, 1], layout=Cc (0x5), const ndim=2

In [27]:
let x_inputs = array![[0.3, 0.7]];
x_inputs

[[0.3, 0.7]], shape=[1, 2], strides=[2, 1], layout=CFcf (0xf), const ndim=2